#### 제주도 핫플레이스 인스타그램에서 크롤링하기

##### matplotlib 한글설정

##### 셀레니움 설정

In [8]:
import matplotlib
# from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc, rcParams
import seaborn as sns

font_path = 'C:/Windows/Fonts/NanumGothic.ttf'
font = font_manager.FontProperties(fname=font_path).get_name() # 'NanumGothic'
rc('font', family=font)
rcParams['axes.unicode_minus'] = False 

In [10]:
# 셀레니움
# 모듈 임포트
from selenium import webdriver

driver = webdriver.Chrome()
url = 'https://www.instagram.com/explore/tags/%EC%A0%9C%EC%A3%BC%EB%8F%84%EB%A7%9B%EC%A7%91/'
driver.get(url)

##### 첫번째 게시글 찾아서 클릭하기


In [11]:
from selenium.webdriver.common.by import By
import time

def select_first_post(driver):
    first = driver.find_elements(By.CLASS_NAME, '_aagw')[0]
    first.click()
    time.sleep(3)

select_first_post(driver)

##### 게시글 정보 가져오기

In [12]:
import re # RegularExpression 정규식 처리 모듈
from bs4 import BeautifulSoup
import unicodedata  # 유니코드 데이터 처리 모듈

def get_content(driver):
    # 1. 데이터가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 2. 본문내용 찾아오기
    try:
        content = soup.select('div._a9zr')[0].text   # _a9zr은 추후 변경됨(인스타그램 정책)
    except:
        content = ''

    # 3. 본문내용에서 해시태그 가져오기(정규식 활용)
    tags = re.findall(r'#[^\s#,\\]+', content)

    # 4. 작성일자 정보
    try:
        date = soup.select('time._aaqe')[0]['datetime'][:10] # _aaqe은 추후 변경됨(인스타그램 정책)
    except:
        date = ''

    # 5. 좋아요수 가져오기
    try:
        # x193iq5w등은 추후 변경됨(인스타그램 정책)
        like = int(soup.select('span.x193iq5w.xeuugli.x1fj9vlw.x13faqbe.x1vvkbs.xt0psk2.x1i0vuye.xvs91rp > span')[0].text)
    except:
        like = 0

    # 6. 위치정보
    # 정규식 학습필수
    # _aagv 마지막 인덱스를 찾으면 된다
    # imgs = soup.select('div._aatk._aatl._aatm div._aagv > img')[0]['alt']
    address = ''
    try:
        regex = r'(\w+[시,읍]\s*)?' \
                r'?(\w+\d*\w*[로,리,길]\s*)' \
                r'?(\w*\d+-?\d*)?'

        place = re.findall(regex, content)
        for pl in place:
            if pl[0] != '' and pl[1] != '' and pl[2] != '':
                address = f'{pl[0]}{pl[1]}{pl[2]}'
    except:
        address = ''

    # 7. 정보 저장
    data = [content, date, like, address, tags]
    return data

get_content(driver)


['mou__9.New arrival🌿[Outf 프리미엄 마레 맨투맨]과하지 않은 오버핏으로 멋스러워요.흔하지 않은 실루엣과 숨은 디테일 부자ㅎ유연한 소재감으로 착용 시 편안해요.맨투맨이지만 고급스러운 느낌.소매를 들면 보이는 구조라 이너를 신경써야 하지만팔도 얇아 보이고 길어 보여요^^컬러별 소장템이예요., 속닥속닥😆Fabric.cottonColor.아이보리 그레이 베이지 소라 블랙Size.총장50 가슴69 암홀20☑️스토어팜 최상단에 전체 상품[신상품순] 카테고리를 클릭하시면,새로 업데이트 된 신상품들을 한눈에 보실 수 있어요.또는 검색창에 상품명 중 한 단어를 검색하시면 제품을 쉽게 찾으실 수 있어요!.수정됨\xa0·\xa01일',
 '2023-06-29',
 54,
 '',
 []]

##### 다음 게시글 열기

In [13]:
def move_next(driver):
    right = driver.find_element(By.CSS_SELECTOR, ('._aaqg._aaqh'))
    right.find_element(By.XPATH, '//button')
    right.click()
    time.sleep(3)

move_next(driver)

##### 자동 크롤링 구현

In [14]:
# 모듈 임포트 생략

driver = webdriver.Chrome()
driver.get('https://www.instagram.com')
time.sleep(2.0)

In [15]:
# 아이디, 패스워드 입력해서 자동 로그인하기
id = 'personar95@gmail.com'
input_id = driver.find_elements(By.CSS_SELECTOR, '._aa4b._add6._ac4d')[0]
input_id.clear()
input_id.send_keys(id)
password = '9876543210'
input_pw = driver.find_elements(By.CSS_SELECTOR, '._aa4b._add6._ac4d')[1]
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()
time.sleep(2.0)

In [16]:
# 알림설정 버튼 클릭
alarm_btn = driver.find_element(By.CSS_SELECTOR, '._a9--._a9_0')
alarm_btn.click()

In [17]:
# 제주도맛집 찾기 함수
def insta_searching(word):
    url = f'https://www.instagram.com/explore/tags/{word}'
    return url

In [18]:
# 크롤링 시작하기
word = '제주도맛집'
url = insta_searching(word)
driver.get(url)
time.sleep(10.0)

select_first_post(driver)

results = []

target = 3
count = 0
for i in range(target):
    count += 1
    try:
        data = get_content(driver)
        results.append(data)
        move_next(driver)
    except:
        time.sleep(2.0)
        move_next(driver)
    print(count)

print(len(results))

1
2
3
3


In [19]:
import pandas as pd
df = pd.DataFrame(results)
df

,0,1,2,3,4
0,er.__bet._trap(광고)제주도에서 쓰끼다시가 가장 잘나오고 서귀포에서 오션...,2023-06-30,0,서귀포시 대포로 16111,"[#서귀포맛집, #제주중문맛집, #서귀포횟집, #중문횟집, #제주도횟집, #중문맛집..."
1,happi_yyyyy여기 가려고 비행기 티켓 당장 끊을 만한 곳✈️#내돈내산 #수미...,2023-06-27,985,구좌읍 구좌해안로 35,"[#내돈내산, #수미언니_제주제주에서, #동복해녀잠수촌, #회국수, #해산물, #제..."
2,mou__9.New arrival🌿[Outf 프리미엄 마레 맨투맨]과하지 않은 오버...,2023-06-29,54,,[]


In [63]:
## 방법찾음!!
import re

# 다시테스트
# 1. 데이터가져오기
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# 2. 본문내용 찾아오기
try:
    # content = soup.select('div._a9zr')[0].text 원본
    content = soup.select('div._a9zr')[0].get_text(separator='|')  # _a9zr은 추후 변경됨(인스타그램 정책)
except:
    content = ''

# 3. 본문내용에서 해시태그 가져오기(정규식 활용)
tags = re.findall(r'#[^\s#,\\]+', content)

# 4. 작성일자 정보
try:
    date = soup.select('time._aaqe')[0]['datetime'][:10] # _aaqe은 추후 변경됨(인스타그램 정책)
except:
    date = ''

# 5. 좋아요수 가져오기
try:
    # x193iq5w등은 추후 변경됨(인스타그램 정책)
    like = int(soup.select('span.x193iq5w.xeuugli.x1fj9vlw.x13faqbe.x1vvkbs.xt0psk2.x1i0vuye.xvs91rp > span')[0].text)
except:
    like = 0

# 6. 위치정보
# 정규식 학습필수
# _aagv 마지막 인덱스를 찾으면 된다
# imgs = soup.select('div._aatk._aatl._aatm div._aagv > img')[0]['alt']
address = ''
try:
    regex = r'(\w+[시,읍]\s*)?' \
            r'?(\w+\d*\w*[로,리,길]\s*)' \
            r'?(\w*\d+-?\d*)?'

    place = re.findall(regex, content)
    for pl in place:
        if pl[0] != '' and pl[1] != '' and pl[2] != '':
            address = f'{pl[0]}{pl[1]}{pl[2]}'
except:
    address = ''

# 7. 정보 저장
temp = soup.select('div._a9zr')[0].get_text(separator='|')

phone_number_list = re.compile('\d{2,3}-\d{3,4}-\d{4}').finditer(temp)
for phone_number in phone_number_list: # 전화번호와 인덱스를 출력한다.
	print(phone_number.group(), phone_number.span())    
 
address

064-799-9229 (328, 340)


'애월읍 애월해안로 97'

In [206]:
import pandas as pd

results_df = pd.DataFrame(results)
results_df.columns = ['content','data','like','place','tags']
results_df.to_excel('./1_crawling_jejudoHotplaces.xlsx', index=False)